In [23]:
import os
import json
import logging
from datetime import datetime

In [24]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [25]:
boto3.set_stream_logger(name="botocore.credentials", level=logging.WARNING)


In [ ]:
def check_aws_configuration():
    try:
        # Attempt to create an AWS S3 client
        s3 = boto3.client('s3')
        
        # List buckets to verify AWS credentials and region
        buckets = s3.list_buckets()
        
        print("AWS Configuration is successful.")
        print("Available S3 Buckets:")
        for bucket in buckets['Buckets']:
            print(f"- {bucket['Name']}")
        
        return True
    except Exception as e:
        print(f"Error: {e}")
        print("AWS Configuration is not successful.")
        return False

# Check AWS configuration
check_aws_configuration()

In [27]:
# sess = sagemaker.Session()
sess = sagemaker.Session(boto_session=boto3.Session(region_name='us-east-1'))
region = sess.boto_region_name
print(region)

us-east-1


In [28]:
role_arn = "arn:aws:iam::173829655778:role/sage_make_full_access"

In [29]:
bucket_name = "imagetaggingdemo"
data_path = "img_clf_multilabel_lst"

nclasses = 3
nimgs_train = 11
nepochs = 10
mini_batch_size = 3

train_instance_type = "ml.g4dn.xlarge"
job_name_prefix = "img-clf-multilabel"

In [30]:
train_image_uri = sagemaker.image_uris.retrieve(
    framework="image-classification",
    region=region,
    image_scope="training",
    version="latest",
)
print(train_image_uri)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


811284229777.dkr.ecr.us-east-1.amazonaws.com/image-classification:1


In [31]:
s3_output_path = "s3://{}/{}/{}".format(bucket_name, data_path, "model_output")


In [32]:
clf_estimator = sagemaker.estimator.Estimator(
    image_uri=train_image_uri,
    role=role_arn,
    instance_count=1,
    instance_type=train_instance_type,
    volume_size=50,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_path,
    sagemaker_session=sess,
)

In [34]:
clf_estimator.set_hyperparameters(
    num_classes=nclasses,  # update this
    epochs=nepochs,  # update this
    num_training_samples=nimgs_train,  # update this
    mini_batch_size=mini_batch_size,  # update this
    num_layers=18,
    use_pretrained_model=1,
    image_shape="3,224,224",
    resize=256,
    learning_rate=0.001,
    use_weighted_loss=1,
    augmentation_type="crop_color_transform",
    precision_dtype="float32",
    multi_label=0,
)

In [35]:
s3_train_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "train_imgs")
s3_valid_imgs = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_imgs")
s3_train_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "train_annots")
s3_valid_annot = "s3://{}/{}/{}".format(bucket_name, data_path, "valid_annots")

train_imgs = sagemaker.inputs.TrainingInput(
    s3_train_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_imgs = sagemaker.inputs.TrainingInput(
    s3_valid_imgs,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
train_annot = sagemaker.inputs.TrainingInput(
    s3_train_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)
valid_annot = sagemaker.inputs.TrainingInput(
    s3_valid_annot,
    distribution="FullyReplicated",
    content_type="application/jpeg",
    s3_data_type="S3Prefix",
)

data_channels = {
    "train": train_imgs,
    "validation": valid_imgs,
    "train_lst": train_annot,
    "validation_lst": valid_annot,
}

In [36]:
timestamp = (
    str(datetime.now().replace(microsecond=0)).replace(" ", "-").replace(":", "-")
)
job_name = job_name_prefix + timestamp
print(job_name)

img-clf-multilabel2023-12-15-15-55-16


In [37]:
clf_estimator.fit(inputs=data_channels, logs=True, job_name=job_name)


INFO:sagemaker:Creating training-job with name: img-clf-multilabel2023-12-15-15-55-16


2023-12-15 10:25:25 Starting - Starting the training job...
2023-12-15 10:25:41 Starting - Preparing the instances for training......
2023-12-15 10:27:22 Downloading - Downloading the training image..................
2023-12-15 10:31:39 Training - Training image download completed. Training in progress.Docker entrypoint called with argument(s): train
Running default environment configuration script
Nvidia gpu devices, drivers and cuda toolkit versions (only available on hosts with GPU):
Fri Dec 15 10:31:52 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.12             Driver Version: 535.104.12   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                         

In [38]:
infer_instance_type = "ml.t2.medium"
model_name = job_name
endpoint_name = job_name

In [39]:
clf_predictor = clf_estimator.deploy(
    initial_instance_count=1,
    instance_type=infer_instance_type,
    endpoint_name=endpoint_name,
    model_name=model_name,
)

INFO:sagemaker:Creating model with name: img-clf-multilabel2023-12-15-15-55-16
INFO:sagemaker:Creating endpoint-config with name img-clf-multilabel2023-12-15-15-55-16
INFO:sagemaker:Creating endpoint with name img-clf-multilabel2023-12-15-15-55-16


---------------------!

In [41]:
sgmkr_runt =  boto3.client("sagemaker-runtime", region_name='us-east-1')


In [51]:
with open("test.jpeg", "rb") as image:
        payload = image.read()
        payload = bytearray(payload)
        
# print(payload)        
response = sgmkr_runt.invoke_endpoint(
    EndpointName = endpoint_name,
    ContentType = 'image/jpeg',
    # Accept = "application/json;verbose",
    Body = payload,
)

prediction = json.loads(response['Body'].read().decode())
print(prediction)

[0.3089269995689392, 0.6767949461936951, 0.014278070069849491]


In [52]:
clf_predictor.delete_endpoint()


INFO:sagemaker:Deleting endpoint configuration with name: img-clf-multilabel2023-12-15-15-55-16
INFO:sagemaker:Deleting endpoint with name: img-clf-multilabel2023-12-15-15-55-16
